# Brain Tumor Detection

### For Google Colab Users

In [ ]:
#@title ## Mount Your Google Drive & Map your Directory
#@markdown Please run this cell (`Ctrl+Enter` or `Shift+Enter`) and follow the steps printed bellow.
import os

from google.colab import drive
drive.mount('/content/gdrive')

# check Google Drive is mounted
if not os.path.isdir("/content/gdrive"):
  raise FileNotFoundError("Your Google Drive isn't mounted. Please run the above cell.")

assignment_dir = "/content/gdrive/MyDrive/workspace/BrainTumorDetection"  #@param{type:"string"}
assignment_dest = "/content/project"

# create symbolic link
!rm -f {assignment_dest}
!ln -s "{assignment_dir}" "{assignment_dest}"
print(f'Succesfully mapped (ln -s) "{assignment_dest}" -> "{assignment_dir}"')

# cd to linked dir
%cd -q {assignment_dest}
print(f'Succesfully changed directory (cd) to "{assignment_dest}"')

Mounted at /content/gdrive
Succesfully mapped (ln -s) "/content/project" -> "/content/gdrive/MyDrive/workspace/BrainTumorDetection"
Succesfully changed directory (cd) to "/content/project"


### Init

In [1]:
#@title ##Init

#@markdown Import required modules and observe your resources

# auto reload
%load_ext autoreload
%autoreload 2

# import
import torch
import torchvision
from PIL import Image
import matplotlib.pyplot as plt
import random
import os
import wandb
wandb.login()

from experiment import Experiment
from post_process import get_masks_from_heatmaps

# Show resources
!nvidia-smi

wandb: Currently logged in as: sgvdan (use `wandb login --relogin` to force relogin)


Wed Jul  7 18:02:35 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.102.04   Driver Version: 450.102.04   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:85:00.0 Off |                    0 |
| N/A   36C    P0    54W / 300W |      0MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Experiment

In [ ]:
# params
config = {
"model_name": "vgg19",
#"model_name": "vgg11",
"data_train_path": os.path.join(".", "Data", "Training"),
"data_test_path": os.path.join(".", "Data", "Testing"),
"epochs": 6,
"train_batch_size": 200,
"lr": 1e-4,
"heatmap_batch_size": 256,
"occlusion_size": (60,60),
"heat_layers" : [['features', layer_num] for layer_num in [0, 2, 5, 7, 10, 12, 14, 16, 19, 21, 23, 25, 28, 30, 32, 34]] +
                [['classifier', -1]],
#"heat_layers" : [0, 3, 6, 8, 11, 13, 16, 18],
"heatmap_threshold" : 0.80,
"shuffle_data": True,
"device": "cuda"
}

with wandb.init(project="BrainTumorDetection", config=config):
    # load best model
    experiment = Experiment(config)
    print("Model's accuracy: ", "{:.2f}".format(experiment.eval_model()), flush=True)
#     experiment.train_model()
#     print("Model's accuracy: ", "{:.2f}".format(experiment.eval_model()), flush=True)
    
    # generate heatmaps
    for i in range(1,5):
        heatmaps = experiment.generate_heatmap(r"./Data/Testing/glioma_tumor/image({idx}).jpg".format(idx=i))
        hot_masks, cold_masks = get_masks_from_heatmaps(heatmaps, thresh=config["heatmap_threshold"], smallest_contour_len=30)
    
    # calculate masks
    

loading vgg19 model...
loading best model state from: ./Models/vgg19.pth
Model's accuracy:  0.98
> /home/labs/waic/sgvdan/workspace/BrainTumorDetection/feature_extractor.py(50)plug_layer()
     48         # Traverse through layers
     49         import pdb; pdb.set_trace()
---> 50         runner = self.model._modules
     51         for key in layer_pos:
     52             runner = runner[key]



ipdb>  n


> /home/labs/waic/sgvdan/workspace/BrainTumorDetection/feature_extractor.py(51)plug_layer()
     49         import pdb; pdb.set_trace()
     50         runner = self.model._modules
---> 51         for key in layer_pos:
     52             runner = runner[key]
     53         layer = runner



ipdb>  n


> /home/labs/waic/sgvdan/workspace/BrainTumorDetection/feature_extractor.py(52)plug_layer()
     50         runner = self.model._modules
     51         for key in layer_pos:
---> 52             runner = runner[key]
     53         layer = runner
     54 



...

TypeError: sequence item 1: expected str instance, int found
> /home/labs/waic/sgvdan/workspace/BrainTumorDetection/feature_extractor.py(55)plug_layer()
     53         layer = runner
     54 
---> 55         hook = self._init_layer_hook('#'.join(layer_pos))
     56         handle = layer.register_forward_hook(hook)
     57         self.layer_hook_handlers.append(handle)



ipdb>  print(layer_pos)


['features', 0]


ipdb>  str(layer_pos)


"['features', 0]"


In [18]:
print(experiment.model._modules['features'])
'features/0'.split('/'), features/1, features/2', 'classifier/-1'

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (17): ReLU(inplace=True)
  (18): MaxPoo